In [1]:
import sys
from os.path import join as pjoin
import platform

# Add the directory for the data and utilities
mat_version = 1196 # what is mat version?

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_root = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2025/"

# Set the directory to load prepared data and utility code
data_dir = pjoin(data_root, f"v1dd_{mat_version}")
utils_dir = pjoin("..", "utils")

# Add utilities to path
sys.path.append(utils_dir)

In [2]:
# Import packages
from caveclient import CAVEclient
import skeleton_plot as skelplot

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csr_array
from typing import Union, Optional
from utils import check_index, adjacencyplot

In [3]:
# Token Generation and Setup
CAVEclient.setup_token("https://global.em.brain.allentech.org")

# Initialize CAVEclient
client = CAVEclient(datastack_name="v1dd_public")

Visit https://global.em.brain.allentech.org/sticky_auth/settings/tokens and copy an existing token or create a new one.


Paste your auth token (input will be hidden):  ········
Set all of your datastacks to use this token and server ('Y', recommended), specify individual datastacks ('n'), or finish now ('exit'). (Y/n/exit)  Y


All datastacks are configured to use this server address.
You will not need to specify a server address when initializing a client for configured datastacks in the future.
Setup complete!


In [6]:
cell_df = pd.read_feather(f"{data_dir}/soma_and_cell_type_{mat_version}.feather")
cell_df['depth_um'] = cell_df['pt_position_trform_y'] / 1_000 # from nm to um
cell_df.head()

,id,pt_position_x,pt_position_y,pt_position_z,pt_position_trform_x,pt_position_trform_y,pt_position_trform_z,pt_root_id,volume,cell_type_coarse,cell_type,depth_um
0,228132,632828,749849,738270,-323721.447979,549910.283106,392909.832613,864691132737039043,458.464831,None,None,549.910283
1,543247,1304922,977915,83880,330339.020171,595962.275760,-306424.551354,864691132730839988,73.345940,None,None,595.962276
2,203262,624680,531094,283770,-252082.627894,203770.728235,21544.029756,864691132654552792,338.276613,E,L3-IT,203.770728
3,350562,894573,478559,163530,20989.259196,117514.626427,-98554.035375,864691132773514104,326.965400,E,L2-IT,117.514626
4,718122,1729859,674111,781200,803635.726721,475075.415268,467669.881328,864691132774106773,333.888647,None,None,475.075415


In [7]:
sub_cell_df = cell_df[cell_df["cell_type_coarse"] == 'E']

In [8]:
sub_cell_df

,id,pt_position_x,pt_position_y,pt_position_z,pt_position_trform_x,pt_position_trform_y,pt_position_trform_z,pt_root_id,volume,cell_type_coarse,cell_type,depth_um
2,203262,624680,531094,283770,-252082.627894,203770.728235,21544.029756,864691132654552792,338.276613,E,L3-IT,203.770728
3,350562,894573,478559,163530,20989.259196,117514.626427,-98554.035375,864691132773514104,326.965400,E,L2-IT,117.514626
10,367611,986529,653935,215865,98861.883924,303237.294062,-68777.223419,864691132777373560,238.028400,E,L4-IT,303.237294
17,525091,1234849,688234,564030,321066.521782,431135.101730,255933.555589,864691132802050269,409.558505,E,L5-IT,431.135102
24,622701,1403163,1019354,230715,391493.580443,684368.956816,-190407.373253,864691132752568948,362.400266,E,L6-CT,684.368957
...,...,...,...,...,...,...,...,...,...,...,...,...
207439,237895,625922,905825,489420,-368197.085320,643194.581600,96360.413891,864691132782578316,248.600654,E,L6-CT,643.194582
207442,546314,1313846,949281,259290,330518.476145,617315.862280,-132311.505385,864691132923394594,363.447329,E,L6-IT,617.315862
207443,48131,349976,576102,285435,-530797.586371,249491.738291,16286.293556,864691132739246420,258.301980,E,L4-IT,249.491738
207446,591233,1433660,685286,71145,547592.387657,292362.112380,-216663.361411,864691132899085806,231.127915,E,L4-IT,292.362112


In [9]:
dendrite_proof_root_ids = np.load(pjoin(data_dir, f'proofread_dendrite_list_{mat_version}.npy'))
axon_proof_root_ids = np.load(pjoin(data_dir, f'proofread_axon_list_{mat_version}.npy'))

In [10]:
sub_cell_axon_proofread_df = sub_cell_df[np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]

In [11]:
sub_cell_axon_proofread_df

,id,pt_position_x,pt_position_y,pt_position_z,pt_position_trform_x,pt_position_trform_y,pt_position_trform_z,pt_root_id,volume,cell_type_coarse,cell_type,depth_um
159,293764,878665,697702,245475,-17654.153169,354663.188122,-51618.075780,864691132625300120,231.369357,E,L4-IT,354.663188
956,295576,842193,737122,325260,-67856.489452,415829.186486,13006.444236,864691132710501931,453.768761,E,L5-IT,415.829186
1991,307952,867646,840563,262935,-69027.465946,503380.292446,-79655.915786,864691132999200949,352.843100,E,L5-IT,503.380292
2312,278794,857790,511462,309780,-18184.720640,190414.097615,49637.501098,864691132585664226,312.136605,E,L3-IT,190.414098
2441,276159,831251,496873,187920,-42822.351357,142725.043006,-74044.776701,864691132698794587,352.059157,E,L2-IT,142.725043
...,...,...,...,...,...,...,...,...,...,...,...,...
205930,397597,1036426,999643,340110,19099.102846,697883.226259,-81961.139771,864691132769830524,392.541011,E,L6-IT,697.883226
206118,294489,823103,635932,272970,-64431.224685,301977.525829,-9080.417561,864691132534315610,236.639903,E,L4-IT,301.977526
206433,385115,975354,850418,394650,17544.597851,553562.857679,34298.147710,864691132679146333,325.794498,E,L6-IT,553.562858
206511,383491,928872,797418,316125,3578.299784,473261.097673,-13876.769010,864691132663975748,457.329970,E,L5-IT,473.261098


In [12]:
ultimate_table = sub_cell_axon_proofread_df[['pt_root_id', 'volume', 'cell_type', 'depth_um']]

In [13]:
# Load synapses 
syn_df = syn_df = pd.read_feather(f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather")

# Load the proofreading info
proofread_ids = np.load(pjoin(data_dir, f"proofread_axon_list_{mat_version}.npy"))

# Mark proofread neurons in the synapse dataframe so we can filter out everything else
syn_df["pre_proofread"] = syn_df["pre_pt_root_id"].isin(proofread_ids)
syn_df["post_proofread"] = syn_df["post_pt_root_id"].isin(proofread_ids)

# These are approximate depth values in post-transformed microns for the pial surface, L1-L2/3 border, L2/3-L4 border, etc. down to the L6-wm border.
layer_bounds = [
    -15,
    91,
    261,
    391,
    537,
    753,
]  

In [14]:
# add the postsynaptic euclidean distance from the presynaptic cell/neuron

def add_euclidean_distance(
    df, 
    x_col1='x1', 
    y_col1='y1', 
    z_col1='z1', 
    x_col2='x2', 
    y_col2='y2', 
    z_col2='z2', 
    new_col='distance'
):
    """
    Calculate Euclidean distance from origin (0,0,0) for x, y, z coordinates in a DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing x, y, z coordinate columns.
    x_col, y_col, z_col : str
        Names of the columns for x, y, z coordinates.
    new_col : str
        Name of the new column to store distances.

    Returns:
    --------
    pandas.DataFrame
        DataFrame with an additional column containing Euclidean distances.
    """
    x = df[x_col1] - df[x_col2]
    y = df[y_col1] - df[y_col2]
    z = df[z_col1] - df[z_col2]
    df[new_col] = np.sqrt(x**2 + y**2 + z**2)
    return df

In [15]:
# copy the synapses dataframe
syn_df_ecldn = syn_df.copy()
syn_df_ecldn.head(5)

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id,pre_proofread,post_proofread
0,354386968,758200.5,802316.1,304380.0,757861.0,802558.6,304650.0,757967.7,802597.4,304380.0,240,864691132536286810,864691132734919083,True,False
1,378070488,792063.2,514342.5,183735.0,792664.6,514284.3,183915.0,792412.4,514294.0,183735.0,3056,864691132572190492,864691132606767301,True,False
2,499493001,977071.3,390075.8,191340.0,976974.3,390104.9,190935.0,976838.5,390337.7,190935.0,1346,864691132573738810,864691132747578447,True,False
3,119675985,444260.0,602544.6,3285.0,443988.4,602311.8,3555.0,444182.4,602370.0,3780.0,3637,864691132572564252,864691132654028028,True,False
4,220616943,574501.9,337249.6,258570.0,574152.7,337016.8,258570.0,574337.0,336900.4,258570.0,420,864691132558380553,864691132828255906,True,False


In [16]:
# compute the euclidean distance for each post_pt_root_id
syn_df_ecldn = add_euclidean_distance( # in um
    syn_df_ecldn, 
    x_col1='post_pt_position_x', 
    y_col1='post_pt_position_y', 
    z_col1='post_pt_position_z',
    x_col2='pre_pt_position_x', 
    y_col2='pre_pt_position_y', 
    z_col2='pre_pt_position_z',
    new_col='euclidean_distance'
)
syn_df_ecldn.head()

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id,pre_proofread,post_proofread,euclidean_distance
0,354386968,758200.5,802316.1,304380.0,757861.0,802558.6,304650.0,757967.7,802597.4,304380.0,240,864691132536286810,864691132734919083,True,False,496.957242
1,378070488,792063.2,514342.5,183735.0,792664.6,514284.3,183915.0,792412.4,514294.0,183735.0,3056,864691132572190492,864691132606767301,True,False,630.451584
2,499493001,977071.3,390075.8,191340.0,976974.3,390104.9,190935.0,976838.5,390337.7,190935.0,1346,864691132573738810,864691132747578447,True,False,417.469532
3,119675985,444260.0,602544.6,3285.0,443988.4,602311.8,3555.0,444182.4,602370.0,3780.0,3637,864691132572564252,864691132654028028,True,False,448.176751
4,220616943,574501.9,337249.6,258570.0,574152.7,337016.8,258570.0,574337.0,336900.4,258570.0,420,864691132558380553,864691132828255906,True,False,419.686168


In [17]:
syn_df_ecldn_ = syn_df_ecldn[syn_df_ecldn['pre_proofread'] * syn_df_ecldn['post_proofread']]

/tmp/ipykernel_4063/2139704821.py:1: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  syn_df_ecldn_ = syn_df_ecldn[syn_df_ecldn['pre_proofread'] * syn_df_ecldn['post_proofread']]


In [18]:
syn_df_ecldn_

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id,pre_proofread,post_proofread,euclidean_distance
22,461895313,917998.3,772740.8,284220.0,918376.6,772818.4,283815.0,918202.0,773080.3,284085.0,540,864691132557909513,864691132931817448,True,True,559.604905
25,301049004,679950.6,813121.9,283050.0,679552.9,812753.3,283095.0,679601.4,812782.4,283140.0,958,864691132578625044,864691132931817448,True,True,544.110513
29,436596114,879663.9,750576.3,347715.0,879266.2,750653.9,348120.0,879469.9,750634.5,347940.0,266,864691132557909513,864691132931817448,True,True,572.897940
45,418665829,860108.7,649153.1,370755.0,860477.3,648978.5,370395.0,860118.4,649182.2,370530.0,381,864691132572564252,864691132833862549,True,True,544.013897
50,363374679,781538.7,726132.3,241110.0,781257.4,726268.1,240480.0,781305.9,725986.8,240705.0,396,864691132578625044,864691132786109590,True,True,703.186554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849355,392461779,810570.8,718508.1,268200.0,810881.2,718634.2,268425.0,810726.0,718731.2,268155.0,487,864691132831272057,864691132713081093,True,True,403.576969
2849379,405769496,830717.7,713939.4,250200.0,830950.5,714414.7,250605.0,831018.4,714298.3,250605.0,155,864691132831272057,864691132656664944,True,True,666.431489
2849391,441048774,873824.5,722097.1,241875.0,873669.3,722271.7,242235.0,874008.8,721951.6,242325.0,176,864691132831272057,864691132672733653,True,True,429.152887
2849402,427402129,846780.9,706460.7,273150.0,846751.8,706344.3,273105.0,846771.2,706412.2,273060.0,574,864691132831272057,864691132741385305,True,True,128.143552


In [19]:
avg_ecldn_dist = (
    syn_df_ecldn_
    .groupby("post_pt_root_id")["euclidean_distance"].mean()
    .reset_index()
    .rename(columns={"post_pt_root_id": "pt_root_id"})
)

In [20]:
avg_ecldn_dist

,pt_root_id,euclidean_distance
0,864691132534275418,543.860992
1,864691132534315610,518.750960
2,864691132535664474,509.823242
3,864691132536286810,531.231788
4,864691132536904794,543.334715
...,...,...
1186,864691133311980112,498.663287
1187,864691133312850768,544.362233
1188,864691133313558608,525.727037
1189,864691133313636944,511.413401


In [21]:
ultimate_table = pd.merge(ultimate_table, avg_ecldn_dist, on=["pt_root_id"])

In [22]:
sum_size = (
    syn_df_ecldn_
    .groupby("post_pt_root_id")["size"].sum()
    .reset_index()
    .rename(columns={"post_pt_root_id": "pt_root_id"})
)

In [23]:
ultimate_table = pd.merge(ultimate_table, sum_size, on=["pt_root_id"])

In [24]:
ultimate_table

,pt_root_id,volume,cell_type,depth_um,euclidean_distance,size
0,864691132625300120,231.369357,L4-IT,354.663188,521.169523,77864
1,864691132710501931,453.768761,L5-IT,415.829186,521.976069,119370
2,864691132999200949,352.843100,L5-IT,503.380292,536.196273,76137
3,864691132585664226,312.136605,L3-IT,190.414098,537.990081,218644
4,864691132698794587,352.059157,L2-IT,142.725043,557.614225,177869
...,...,...,...,...,...,...
669,864691132769830524,392.541011,L6-IT,697.883226,533.836972,62380
670,864691132534315610,236.639903,L4-IT,301.977526,518.750960,110487
671,864691132679146333,325.794498,L6-IT,553.562858,544.810752,53517
672,864691132663975748,457.329970,L5-IT,473.261098,508.778485,87316


In [26]:
connectivity_ct_df = syn_df_ecldn_.merge(
    cell_df[["pt_root_id", "cell_type_coarse", "cell_type"]].rename(
        columns={
            "pt_root_id": "pre_pt_root_id",
            "cell_type": "cell_type_pre",
            "cell_type_coarse": "ei_pre",
        }
    ),
    on="pre_pt_root_id",
).merge(
    cell_df[["pt_root_id", "cell_type_coarse", "cell_type"]].rename(
        columns={
            "pt_root_id": "post_pt_root_id",
            "cell_type": "cell_type_post",
            "cell_type_coarse": "ei_post",
        }
    ),
    on="post_pt_root_id",
)

In [27]:
connectivity_ct_df

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id,pre_proofread,post_proofread,euclidean_distance,ei_pre,cell_type_pre,ei_post,cell_type_post
0,461895313,917998.3,772740.8,284220.0,918376.6,772818.4,283815.0,918202.0,773080.3,284085.0,540,864691132557909513,864691132931817448,True,True,559.604905,I,DTC,E,L5-ET
1,301049004,679950.6,813121.9,283050.0,679552.9,812753.3,283095.0,679601.4,812782.4,283140.0,958,864691132578625044,864691132931817448,True,True,544.110513,I,DTC,E,L5-ET
2,436596114,879663.9,750576.3,347715.0,879266.2,750653.9,348120.0,879469.9,750634.5,347940.0,266,864691132557909513,864691132931817448,True,True,572.897940,I,DTC,E,L5-ET
3,363374679,781538.7,726132.3,241110.0,781257.4,726268.1,240480.0,781305.9,725986.8,240705.0,396,864691132578625044,864691132786109590,True,True,703.186554,I,DTC,I,PTC
4,491726200,962385.5,676759.3,316035.0,962210.9,677069.7,316620.0,962366.1,676856.3,316305.0,442,864691132562471396,864691132773784759,True,True,684.878325,I,DTC,E,L4-IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197830,392461779,810570.8,718508.1,268200.0,810881.2,718634.2,268425.0,810726.0,718731.2,268155.0,487,864691132831272057,864691132713081093,True,True,403.576969,I,PTC,I,PTC
197831,405769496,830717.7,713939.4,250200.0,830950.5,714414.7,250605.0,831018.4,714298.3,250605.0,155,864691132831272057,864691132656664944,True,True,666.431489,I,PTC,E,L5-IT
197832,441048774,873824.5,722097.1,241875.0,873669.3,722271.7,242235.0,874008.8,721951.6,242325.0,176,864691132831272057,864691132672733653,True,True,429.152887,I,PTC,E,L4-IT
197833,427402129,846780.9,706460.7,273150.0,846751.8,706344.3,273105.0,846771.2,706412.2,273060.0,574,864691132831272057,864691132741385305,True,True,128.143552,I,PTC,E,L5-IT


In [28]:
# find I to E connections

connectivity_ct_IE = connectivity_ct_df.query('ei_pre == "I" and ei_post == "E"')
connectivity_ct_IE.head()

,id,pre_pt_position_x,pre_pt_position_y,pre_pt_position_z,post_pt_position_x,post_pt_position_y,post_pt_position_z,ctr_pt_position_x,ctr_pt_position_y,ctr_pt_position_z,size,pre_pt_root_id,post_pt_root_id,pre_proofread,post_proofread,euclidean_distance,ei_pre,cell_type_pre,ei_post,cell_type_post
0,461895313,917998.3,772740.8,284220.0,918376.6,772818.4,283815.0,918202.0,773080.3,284085.0,540,864691132557909513,864691132931817448,True,True,559.604905,I,DTC,E,L5-ET
1,301049004,679950.6,813121.9,283050.0,679552.9,812753.3,283095.0,679601.4,812782.4,283140.0,958,864691132578625044,864691132931817448,True,True,544.110513,I,DTC,E,L5-ET
2,436596114,879663.9,750576.3,347715.0,879266.2,750653.9,348120.0,879469.9,750634.5,347940.0,266,864691132557909513,864691132931817448,True,True,572.897940,I,DTC,E,L5-ET
4,491726200,962385.5,676759.3,316035.0,962210.9,677069.7,316620.0,962366.1,676856.3,316305.0,442,864691132562471396,864691132773784759,True,True,684.878325,I,DTC,E,L4-IT
5,503206739,982687.6,730885.3,348435.0,982464.5,730274.2,348075.0,982367.5,730429.4,348165.0,416,864691132558474249,864691132941150385,True,True,743.516523,I,PTC,E,L5-IT


In [29]:
# Count the number of each inhibtory presynaaptic cell type connection

# query dataframe to count the number of inhibitory cell connections
queried_connectivity_ct_IE = (
    connectivity_ct_IE.groupby(["post_pt_root_id","cell_type_pre"])
    .agg(num_connections=("cell_type_pre", "count"))
    .reset_index()  
    .rename(columns={"post_pt_root_id": "pt_root_id"})
)

queried_connectivity_ct_IE.head()

,pt_root_id,cell_type_pre,num_connections
0,864691132534275418,DTC,30
1,864691132534275418,PTC,9
2,864691132534275418,STC,3
3,864691132534315610,DTC,82
4,864691132534315610,PTC,79


In [30]:
dtc_table = queried_connectivity_ct_IE[queried_connectivity_ct_IE['cell_type_pre'] == 'DTC'][['pt_root_id', 'num_connections']]
itc_table = queried_connectivity_ct_IE[queried_connectivity_ct_IE['cell_type_pre'] == 'ITC'][['pt_root_id', 'num_connections']]
ptc_table = queried_connectivity_ct_IE[queried_connectivity_ct_IE['cell_type_pre'] == 'PTC'][['pt_root_id', 'num_connections']]
stc_table = queried_connectivity_ct_IE[queried_connectivity_ct_IE['cell_type_pre'] == 'STC'][['pt_root_id', 'num_connections']]

In [31]:
ultimate_table = pd.merge(
    ultimate_table, 
    dtc_table.rename(columns={"num_connections": "dtc_num_connections"}),
    on=["pt_root_id"]
)

In [32]:
ultimate_table = pd.merge(
    ultimate_table, 
    itc_table.rename(columns={"num_connections": "tc_num_connections"}),
    on=["pt_root_id"]
)

In [33]:
ultimate_table = pd.merge(
    ultimate_table, 
    ptc_table.rename(columns={"num_connections": "ptc_num_connections"}),
    on=["pt_root_id"]
)

In [34]:
ultimate_table

,pt_root_id,volume,cell_type,depth_um,euclidean_distance,size,dtc_num_connections,tc_num_connections,ptc_num_connections
0,864691132625300120,231.369357,L4-IT,354.663188,521.169523,77864,47,2,56
1,864691132710501931,453.768761,L5-IT,415.829186,521.976069,119370,32,4,105
2,864691132999200949,352.843100,L5-IT,503.380292,536.196273,76137,7,4,68
3,864691132657879768,377.762618,L3-IT,232.846732,520.103258,229979,126,1,105
4,864691132722099342,237.380760,L4-IT,332.657513,522.721010,45052,30,2,34
...,...,...,...,...,...,...,...,...,...
448,864691132774014868,381.188067,L5-ET,440.857171,540.992841,331407,159,22,50
449,864691132769830524,392.541011,L6-IT,697.883226,533.836972,62380,12,2,32
450,864691132679146333,325.794498,L6-IT,553.562858,544.810752,53517,13,3,29
451,864691132663975748,457.329970,L5-IT,473.261098,508.778485,87316,31,9,63
